In [6]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
class value:
    
    def _init_(self, data, children=(), label="", op=""):
        self.data= data
        self.prev=set(children)
        self.grad= 0.0
        self.label=label
        self.op=op
        self._backward= lambda: None
        
    def _repr_(self):
        return f"value(data={self.data})"
    
    def _add_(self, other):
        if isinstance(other, value):
            value(other)
        else:
            other=other
        
        out = value(self.data + other.data, (self,other), "+")
        
        def _backward():
            self.grad+=out.grad
            other.grad+=out.grad
            
        out._backward=_backward
                
        return out
    
    def _radd_(self, other):
        return other+self
    
    def _mul_(self, other):
        if isinstance(other, value):
            value(other)
        else:
            other=other
        
        out = value(self.data * other.data, (self, other), "*")
        
        def _backward():
            self.grad+=other.data*out.grad
            other.grad+=self.grad*out.grad
        
        out._backward=_backward
            
        return out
    
    def _rmul_(self, other):
        return other*self
    
    def _neg_(self):
        return self*-1
    
    def _subt_(self,other):
        return self + (-other)
    
    def _pow_(self, other):
        assert isinstance(other, (int,float))
        out= value(self.data**other, (self,), f'**{other}' )
        
        def _backward():
            self.grad+=other * (self**(other-1)) * out.grad
            
        out._backward=_backward

        return out 
    
    def _truediv_(self, other):
        return self * other**-1
    
    def _exp_(self):
        out= value(math.exp(self.data), (self,), "exp")
        
        def _backward():
            self.grad+=out.data*out.grad
            
        out._backward=_backward
        
        return out

    def tanh(self):
        x=self.data
        ans=(math.exp(2*x)-1)/(math.exp(2*x)+1)
        out=value(ans, (self,), "tanh")
        
        def _backward():
            self.grad+=(1-ans**2)*out.grad
            
        out._backward=_backward
        
        return out
    
    def backward(self):
        topo = []
        visited=set()
        
        def build_topo(neuron):
            if neuron not in visited:
                visited.add(neuron)
                for child in neuron.prev:
                    build_topo(child)
                topo.append(neuron)
                
        build_topo(self)
        
        self.grad=1.0
        for node in reversed(topo):
            node._backward()

In [8]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [12]:
class neuron:
    def _init_(self, num_inp):
        self.w= [value(random.uniform(-1,1)) for i in range (num_inp)]
        self.b= value(random.uniform(-1,1))
        
    def _call_(self, x): #wx+b
        activation=self.b
        for wi,xi in zip(self.w, x):
            activation+= wi*xi
            
        out= activation.tanh()
        return out
    
    def parameters(self):
        return self.w+[self.b]
    
class layer:
    def _init_(self, num_imp, num_out):
        self.neurons= [neuron(num_imp) for i in range (num_out)]
        
    def _call_(self, x):
        out= [neuron(x) for neuron in self.neurons]
        return out
    
    def parameters(self):
        params=[]
        for neuron in self.neurons:
            params.extend(neuron.parameters)
        return params
    
class MLP:
    def _init_(self, num_inp, nums_out):
        sizes= [num_inp]+nums_out
        self.layers= [layer(sizes[i], sizes[i+1]) for i in range (len(nums_out))]
        
    def _call_(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        params=[]
        for layer in self.layers:
            params.extend(layer.parameters)
        return params
